In [ ]:
import torch
import numpy as np
import torch.nn as nn
from scipy.io import loadmat
import matplotlib.pyplot as plt
from Dataset import Dataset, ELM, device
torch.set_default_dtype(torch.float64)

In [ ]:
train = loadmat('data_training_reactor_pol.mat')
test = loadmat('data_validation_reactor_pol.mat')

y_train = 0.0001*(train['NAMW'][0]-train['NAMW'][0][0])
y_test = 0.0001*(test['NAMW'][0]-test['NAMW'][0][0])

x_train = np.concatenate((100*np.concatenate(([[0]], (train['FI'].T[1:]-train['FI'].T[0])), axis=0),
                          100*np.concatenate(([[0], [0]], (train['FI'].T[2:]-train['FI'].T[0])), axis=0),
                          np.concatenate(([0], y_train[1:]), axis=0).reshape(-1, 1),
                          np.concatenate(([0, 0], y_train[2:]), axis=0).reshape(-1, 1)), axis=1)

x_test = np.concatenate((100*np.concatenate(([[0]], (test['FI'].T[1:]-test['FI'].T[0])), axis=0),
                         100*np.concatenate(([[0], [0]], (test['FI'].T[2:]-test['FI'].T[0])), axis=0),
                         np.concatenate(([0], y_test[1:]), axis=0).reshape(-1, 1),
                         np.concatenate(([0, 0], y_test[2:]), axis=0).reshape(-1, 1)), axis=1)

In [ ]:
trainset = Dataset(x_train, y_train)
testset = Dataset(x_test, y_test)

activation = nn.Sigmoid()

In [ ]:
Lmax, epsilon = 1500, 1e-7
E, L = trainset[:][1], 0
k = 5 # 1 dla I-ELM, > 1 dla EI-ELM
temp_model = ELM(4, Lmax, 1).to(device)
Error = [torch.norm(E)]

while L < Lmax and torch.norm(E) > epsilon:
    L += 1
    Enorm = torch.norm(E)
    for i in range(k):
        W1 = torch.randn(5, 1).to(device)
        h = activation(W1[0].reshape(-1, 1) + W1[1:].T @ trainset[:][0].T.to(device))
        beta = torch.div(E.reshape(1, -1).to(device) @ h.T, h @ h.T).to(device)
        if Enorm > torch.norm(E.to(device) - beta @ h) or L == 1:
            h_star = h.to(device)
            W1_star = W1.to(device)
            beta_star = beta.to(device)
            Enorm = torch.norm(E.to(device) - beta @ h)
    E = E.to(device) - beta_star @ h_star
    Error.append(torch.norm(E.cpu()))
    temp_model.fc1.bias.data[L-1], temp_model.fc1.weight.data[L-1], temp_model.fc2.weight.data[:,L-1] = W1_star[0], W1_star[1], beta_star

plt.plot(Error)
plt.plot([epsilon]*len(Error), 'r--')
plt.grid()
plt.show()

In [ ]:
model = ELM(4, L, 1).to(device)

model.fc1.bias.data, model.fc1.weight.data, model.fc2.weight.data = \
temp_model.fc1.bias.data[:L], temp_model.fc1.weight.data[:L], temp_model.fc2.weight.data[:L]

In [ ]:
print(f'Train error: {(model(trainset[:][0].clone().detach().reshape(-1, 4).to(device))-trainset[:][1].reshape(-1, 1).to(device)).reshape(-1).pow(2).mean()}')
print(f'Test error: {(model(testset[:][0].clone().detach().reshape(-1, 4).to(device))-testset[:][1].reshape(-1, 1).to(device)).reshape(-1).pow(2).mean()}')
plt.plot(y_test)
plt.plot(model(torch.tensor(x_test).reshape(-1, 4).to(device)).cpu().detach().numpy(), 'r--')
plt.legend(['Dane weryfikujące', 'Model'], loc='upper right')
plt.xlabel('x')
plt.ylabel('y')
plt.grid()
plt.show()

In [ ]:
plt.plot([np.min(y_test), np.max(y_test)], [np.min(y_test), np.max(y_test)])
plt.plot(y_test, model(torch.tensor(x_test).reshape(-1, 4).to(device)).cpu().detach().numpy(), '.', markersize=1)
plt.grid()
plt.show()